<a href="https://colab.research.google.com/github/andry2327/Machine-Learning-and-Deep-Learning-LAB--Google-Colab---PoliTo/blob/main/L05/es1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision import transforms as T
import torch.nn.functional as F

Step 2: define AlexNet


In [ ]:
class AlexNet(torch.nn.Module):
  def __init__(self, num_classes=10):
    super(AlexNet, self).__init__()
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
        torch.nn.BatchNorm2d(96),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 3, stride = 2))
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
        torch.nn.BatchNorm2d(256),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 3, stride = 2))
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
        torch.nn.BatchNorm2d(384),
        torch.nn.ReLU())
    self.layer4 = torch.nn.Sequential(
        torch.nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
        torch.nn.BatchNorm2d(384),
        torch.nn.ReLU())
    self.layer5 = torch.nn.Sequential(
        torch.nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
        torch.nn.BatchNorm2d(256),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 3, stride = 2))
    self.fc = torch.nn.Sequential(
        torch.nn.Dropout(0.5),
        torch.nn.Linear(9216, 4096),
        torch.nn.ReLU())
    self.fc1 = torch.nn.Sequential(
        torch.nn.Dropout(0.5),
        torch.nn.Linear(4096, 4096),
        torch.nn.ReLU())
    self.fc2= torch.nn.Sequential(
        torch.nn.Linear(4096, num_classes))
    
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    out = self.fc1(out)
    out = self.fc2(out)
    return out

Step 3: define the loss function


In [ ]:
def get_loss_function():
  loss_function = torch.nn.CrossEntropyLoss()
  return loss_function

Step 4: define the optimizer

In [ ]:
def get_optimizer(net, lr, wd, momentum):
    optimizer = torch.optim.SGD(params=net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
    return optimizer

Step 5: train function

In [ ]:
def train(net, data_loader, optimizer, cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
    cumulative_accuracy = 0.
    net.train() # Strictly needed if network contains layers which has different behaviours between train and test
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        # Load data into GPU
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs) # Forward pass
        loss = cost_function(outputs, targets) # Apply the loss
        loss.backward() # Backward pass
        optimizer.step() # Update parameters
        optimizer.zero_grad() # Reset the optimizer
        samples += inputs.shape[0]
        cumulative_loss += loss.item()
        _, predicted = outputs.max(1)
        cumulative_accuracy += predicted.eq(targets).sum().item()
    return cumulative_loss/samples, cumulative_accuracy/samples*100

Step 6: test function

In [ ]:
def test(net, data_loader, cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
    cumulative_accuracy = 0.
    net.eval() # Strictly needed if network contains layers which have different behaviours between train and test
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            # Load data into GPU
            samples += len(inputs)
            inputs, targets = inputs.to(device), targets.to(device)
            # Forward pass
            outputs = net(inputs)
            _, predicted = outputs.max(1)
            cumulative_accuracy += predicted.eq(targets).sum().item()
    return cumulative_loss/samples, cumulative_accuracy/samples*100

Step 7: loading the data

In [ ]:
def get_data(batch_size, test_batch_size=256):
    # Prepare data transformations and then combine them sequentially
    transform = list()
    transform.append(T.Resize((227,227)))
    transform.append(T.ToTensor()) # Converts Numpy to Pytorch Tensor
    transform.append(T.Normalize(mean=[0.5], std=[0.5])) # Normalizes the Tensors between [-1, 1]
    transform = T.Compose(transform) # Composes the above transformations into one.
    # Load data
    full_training_data = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
    test_data = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True)
    # Create train and validation splits
    num_samples = len(full_training_data)
    training_samples = int(num_samples*0.8+1)
    validation_samples = num_samples - training_samples
    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples,
    validation_samples])
    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

# Main

In [ ]:
''' Input arguments batch_size: 

Size of a mini-batch device: GPU where you want to train your network 
weight_decay: Weight decay co-efficient for regularization of weights 
momentum: Momentum for SGD optimizer 
epochs: Number of epochs for training the network 

'''

def main(batch_size=128, device='cuda:0', learning_rate=0.05, weight_decay=0.000001, momentum=0.9, epochs=20):
    train_loader, val_loader, test_loader = get_data(batch_size)
    # TODO for defining AlexNet
    net = AlexNet().to(device)
    optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
    loss_function = get_loss_function()

    for e in range(epochs):

        train_loss, train_accuracy = train(net, train_loader, optimizer, loss_function)
        val_loss, val_accuracy = test(net, val_loader, loss_function)

        print('Epoch: {:d}'.format(e+1))
        print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
        print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
        print('-----------------------------------------------------')
        print('After training:')

        train_loss, train_accuracy = test(net, train_loader, loss_function)
        val_loss, val_accuracy = test(net, val_loader, loss_function)
        test_loss, test_accuracy = test(net, test_loader, loss_function)

        print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
        print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
        print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
        print('-----------------------------------------------------')

In [ ]:
main()

Files already downloaded and verified
Files already downloaded and verified
Epoch: 1
	 Training loss 0.01289, Training accuracy 39.16
	 Validation loss 0.00000, Validation accuracy 46.43
-----------------------------------------------------
After training:
	 Training loss 0.00000, Training accuracy 46.71
	 Validation loss 0.00000, Validation accuracy 46.43
	 Test loss 0.00000, Test accuracy 47.38
-----------------------------------------------------
Epoch: 2
	 Training loss 0.01010, Training accuracy 52.94
	 Validation loss 0.00000, Validation accuracy 57.60
-----------------------------------------------------
After training:
	 Training loss 0.00000, Training accuracy 58.19
	 Validation loss 0.00000, Validation accuracy 57.60
	 Test loss 0.00000, Test accuracy 57.34
-----------------------------------------------------
Epoch: 3
	 Training loss 0.00832, Training accuracy 62.34
	 Validation loss 0.00000, Validation accuracy 63.58
-----------------------------------------------------
Aft